## 反射

运行时获取类的信息

In [1]:
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def print(self,x,y):
        print(x,y)

In [2]:
p = Point(3,5)

In [3]:
p.x

3

In [4]:
p.y

5

In [7]:
p.print(6,8)

6 8


In [8]:
p.__dict__       ## 获取信息

{'x': 3, 'y': 5}

In [9]:
p.__dict__['z'] = 10         ##  这其实就是反射的体现   等价于 p.z = 10

In [10]:
p.z

10

In [11]:
p.__dict__

{'x': 3, 'y': 5, 'z': 10}

In [12]:
dir(p)      ## 看到所有的属性及方法

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'print',
 'x',
 'y',
 'z']

介绍三个内置函数

### getattr   setattr   hasattr

* 通过成员名称，获取对象的成员

In [13]:
p.__dict__['x']    ## 对于属性来说，可以通过__dict__ 获取     注：仅对于属性 

3

In [14]:
p.__dict__['y']

5

方法怎么办呢？比如print方法

In [15]:
p.__dict__        ## __dict__里看不到方法

{'x': 3, 'y': 5, 'z': 10}

In [16]:
getattr(p,'print')

<bound method Point.print of <__main__.Point object at 0x7f74d0351ba8>>

In [18]:
getattr(p,'print')(6,8)         ## ok,成员方法无法通过__dict__获取，但是可以通过getattr函数获取！ 
                                ## 等价于，p.print(6,8)

6 8


In [19]:
getattr(p,'x')      ## getattr 也可以获取到属性    getattr功能更广啦！

3

In [20]:
setattr(p,'haha','abcd')                ## 动态的设置了一个haha属性！ 等价于 p.haha = 'abcd'

In [21]:
p.haha            ## 这就有了haha这个属性啦！

'abcd'

setattr和 getattr 是对立的，一个是设置，一个是获取

In [22]:
hasattr(p,'print')      ## 判断有没有

True

In [23]:
hasattr(p,'not_found')    ## 显然是没有的

False

setattr 可以动态增加方法嘛？

In [24]:
def mm(self):
    print(self.x)

In [25]:
setattr(p,'mm',mm)       ## 现在还关联不过来

In [26]:
p.mm()

TypeError: mm() missing 1 required positional argument: 'self'

setattr 对象是实例，实例是无法动态增加方法的     
经证实，这句话是错的！

通常来说，不会给一个实例动态增加一个方法的

探索性的实验一下：

In [27]:
import types

In [28]:
setattr(p,'mm',types.MethodType(mm,p))

In [29]:
p.mm()

3


In [ ]:
## 以上3是怎么得来了？      ## 前面定义的这个函数(方法)
def mm(self):
    print(self.x)       

验证得到：setattr 对象是实例，实例是无法动态增加方法的  这句话是不对的！

给实例动态增加方法需要先把函数转化为方法，转化的方法  types.MethodType

但是，真的很少给一个实例动态增加一个方法的！！！因为没什么用！很少有这种场景！

为什么要大费周折的这么做呢？而不直接使用或修改呢？

In [ ]:
##  看这样一个类       看看getattr这些都有什么用处

In [30]:
class Command:
    def cmd1(self):
        print('cmd1')
        
    def cmd2(self):
        print('cmd2')
        
    def run(self):
        while True:
            cmd = input('>>>').strip()
            if cmd == 'quit':
                return
            getattr(self,cmd, lambda :print('not found cmd {}'.format(cmd)))()
            ##  上行最后这个()想半天，应该是调用方法的；比如匹配到了cmd2，那么运行cmd2()
            ##  对，就是这个意思！方法需要加括号

In [31]:
cmd = Command()

In [32]:
cmd.run()

>>>cmd1
cmd1
>>>cmd2
cmd2
>>>cmd3
not found cmd cmd3
>>>quit


以上这个，只是个简单的例子，还有RPC的例子

RPC

In [ ]:
POST /service/method

### 好，再来看三个魔术方法

### `__getattr__`  `__setattr__`  `__delattr__`

In [33]:
class A:
    def __init__(self):
        self.x = 3

In [34]:
a = A()

In [35]:
a.x

3

In [36]:
a.y

AttributeError: 'A' object has no attribute 'y'

In [37]:
class A:
    def __init__(self):
        self.x = 3
    def __getattr__(self,name):
        return 'missing property {}'.format(name)

In [38]:
a = A()

In [39]:
a.x

3

In [40]:
a.y

'missing property y'

In [41]:
a.z

'missing property z'

当一个类定义了`__getattr__`方法时，如果访问不存在成员，会调用`__getattr__`方法

In [42]:
class A:
    NAME = 'A'
    
    def __init__(self):
        self.x = 3
        
    def __getattr__(self,name):
        return name

In [43]:
a = A()

In [44]:
a.__dict__            ## dict的

{'x': 3}

In [45]:
a.__class__.__dict__      ## class 的dict的

mappingproxy({'NAME': 'A', '__doc__': None, '__module__': '__main__', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__init__': <function A.__init__ at 0x7f74d03128c8>, '__getattr__': <function A.__getattr__ at 0x7f74d0312840>})

属性的查找顺序

`dict >> class >> __getattr__ `     

如果dict里有，就直接返回了，如果没有就到class的dict里找，如果还没有就找`__getattr__`

In [46]:
a.x      ## 在dict里面

3

In [47]:
a.NAME   ## 在class的里面

'A'

In [48]:
a.y      ## 以上都不在，就走__getattr__

'y'

成员的查找顺序就是这样的！理解了！`__getattr__`就理解了

In [49]:
class A:
    def __init__(self):
        self.x = 3
        
    def __setattr__(self,name,value):
        print('set {} to {}'.format(name,value))

In [50]:
a = A()

set x to 3


In [51]:
a.x           ##  好奇怪，好奇怪，好奇怪！

AttributeError: 'A' object has no attribute 'x'

In [52]:
a.y = 5

set y to 5


In [53]:
a.x = 9

set x to 9


当一个类实现了`__setattr__`时，任何地方对这个类的对象增加属性，或者对现有属性赋值，都会调用`__setattr__`

In [54]:
a.__dict__      ## 里面什么都没有

{}

In [55]:
class A:
    def __init__(self):
        self.x = 3
        
    def __setattr__(self,name,value):
        print('set {} to {}'.format(name,value))
        self.__dict__[name] = value               ## 看看这样OK嘛

In [56]:
a = A()

set x to 3


In [57]:
a.x         ## OK,出来了

3

In [58]:
a.y = 5

set y to 5


In [59]:
a.__dict__

{'x': 3, 'y': 5}

所以，当需要对实例属性的修改，做一些额外操作的时候，可以使用`__setattr__`

cymon说，`__setattr__`不是这么好用，坑很多！非常容易出错，尽量不要使用！

`__delattr__`   这个用的也相对较少！！

In [60]:
class A:
    def __init__(self):
        self.x = 3
        
    def __delattr__(self,name):    ## 当一个实现了__delattr__ 方法时，删除其实例的属性，会调用此方法
        print('you can not delete property: {}'.format (name))

In [61]:
a = A()

In [62]:
a.x

3

In [63]:
del a.x       ##  删除不了了，明显是执行了__delatter__

you can not delete property: x


In [64]:
a.y = 5    ## 即使动态加入的，也是会被保护的

In [65]:
del a.y

you can not delete property: y


这三个方法，`__getattr__` 是用的最多的，`__delattr__`其次，`__setattr__`用的最少！

### 再来一个内置函数

###  `__getattribute__`

In [75]:
class A:
    NAME = 'A'
    
    def __init__(self):
        self.x = 3
    
    def __getattribute__(self,name):
        return 'haha'
    
    def method(self):
        print('method')

In [76]:
a = A()

In [77]:
a.x

'haha'

In [78]:
a.NAME

'haha'

In [79]:
a.__dict__

'haha'

In [80]:
a.method

'haha'

都是返回'haha'，这杀伤力太大了！

因为它具有最高的查找顺序！！！！！

`__getattribute__`  >  `__dict__`  >  `__class__.__dict__`  > `__getattr__`

In [81]:
dir(a)     ## 什么都出不来了！

[]

### `__getattribute__ `   它只是针对实例的

In [82]:
A.NAME    ## 类的还是没问题的！

'A'

`__getattribute__ `  99.9% 的时候，用不到这个！！！

comyn没用过